## Importing of Libraries


In [1]:
import time
from copy import deepcopy

import numpy as np
import torch
from dlordinal.datasets import FGNet
from dlordinal.losses import OrdinalEcocDistanceLoss
from dlordinal.models import resnet18_ecoc
from sklearn.metrics import (accuracy_score, cohen_kappa_score,
                             confusion_matrix, mean_absolute_error)
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
from torch import cuda
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor
from tqdm import tqdm

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the *DataLoader*, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [2]:
optimiser_params = {
    'lr': 1e-3,
    'bs': 200,
    'epochs': 5,
    's': 2,
    'c': 0.2,
    'beta': 0.5
}

workers = 3

Now we use the *FGNet* method to download and preprocess the images. Once that is done with the training data, we create a validation partition comprising 15% of the data using the *StratifiedShuffleSplit* method. Finally, with all the partitions, we load the images using a method called *DataLoader*.

In [3]:
fgnet = FGNet(root="./datasets/fgnet", download=True, process_data=True)

complete_train_data = ImageFolder(
    root="./datasets/fgnet/FGNET/train", transform=Compose([ToTensor()])
)
test_data = ImageFolder(
    root="./datasets/fgnet/FGNET/test", transform=Compose([ToTensor()])
)

num_classes = len(complete_train_data.classes)
classes = complete_train_data.classes
targets = complete_train_data.targets

# Create a validation split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
sss_splits = list(
    sss.split(X=np.zeros(len(complete_train_data)), y=complete_train_data.targets)
)
train_idx, val_idx = sss_splits[0]

# Create subsets for training and validation
train_data = Subset(complete_train_data, train_idx)
val_data = Subset(complete_train_data, val_idx)

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

# Create dataloaders
train_dataloader = DataLoader(
    train_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
val_dataloader = DataLoader(
    val_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
test_dataloader = DataLoader(
    test_data, batch_size=optimiser_params["bs"], shuffle=False, num_workers=workers
)

# Get image shape
img_shape = None
for X, _ in train_dataloader:
    img_shape = list(X.shape[1:])
    break
print(f"Detected image shape: {img_shape}")

# Define class weights for imbalanced datasets
class_weights = class_weight.compute_class_weight(
    "balanced", classes=[int(c) for c in classes], y=targets
)
print(f"{class_weights=}")
class_weights = (
    torch.from_numpy(class_weights).float().to(device)
)  # Transform to Tensor

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cuda device
Detected image shape: [3, 128, 128]
class_weights=array([1.5       , 0.55165289, 1.01136364, 0.84493671, 1.0511811 ,
       2.51886792])


## Model and optimiser

We are using a modified version of the *ResNet* architecture, specifically designed for the loss function explained in the next section. In this case, the *ResNet* model is not pretrained with *ImageNet*, so we will need to undergo an extensive learning process.

To adapt the outputs of the model to this, the final fully-connected block is substituted by $Q-1$ fully-connected blocks [1], each one with a single output unit with sigmoid activation.

As an alternative to the *ResNet* architecture, you can use the *VGG* architecture, which has also been implemented to work with the loss function explained in the following sections.

Finally, we define the *Adam* optimiser, which is used to adjust the network's weights and minimize the error of a loss function.

[1]: Barbero-Gómez, J., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). *Error-correcting output codes in the framework of deep ordinal classification.* Neural Processing Letters, 1-32. Springer.

In [4]:
model = resnet18_ecoc(num_classes=num_classes, activation_function='relu').to(device)

# Optimizer
optimizer = Adam(model.parameters(), lr=optimiser_params['lr'])

## Loss Function

The original $Q$-class ordinal problem is decomposed into $Q-1$ binary decision problems, what is known as Ordinal Binary Decomposition (ODB) [1]. So the categorical cross-entropy has been substituted by the Mean Squared Error loss because it copes better with the distance function used for the Error-Conecting Output Codes (ECOC) decision:

$$
\ell = \frac{1}{N} ∑_{i=1}^N ∑_{k=1}^{Q-1} (\mathbf{1} \{y_i \succ \mathcal{C}_k\} - P(y_i \succ \mathcal{C}_k | x_i))^2
$$

where $\mathbf{1} \{y_i \succ \mathcal{C}_k\}$ is the indicator function that is equal to 1 when $y_i \succ \mathcal{C}_k$ and 0 otherwise, and $P(y_i \succ \mathcal{C}_k | x_i)$ is the probability that $y_i \succ \mathcal{C}_k$ predicted by the network given a sample.

[1]: Barbero-Gómez, J., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). *Error-correcting output codes in the framework of deep ordinal classification.* Neural Processing Letters, 1-32. Springer. doi.org/10.1007/s11063-022-10824-7

In [5]:
loss_fn = OrdinalEcocDistanceLoss(
    num_classes=num_classes, class_weights=class_weights, device=device
)

## Metrics

In [6]:
# Metrics computation


def compute_metrics(y_true: np.ndarray, 
    y_pred: np.ndarray, 
    num_classes: int):

    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)

    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    labels = range(0, num_classes)

    # Metrics calculation
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic', labels=labels)
    ms = minimum_sensitivity(y_true, y_pred, labels=labels)
    mae = mean_absolute_error(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    off1 = accuracy_off1(y_true, y_pred, labels=labels)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

    metrics = {
        'QWK': qwk,
        'MS': ms,
        'MAE': mae,
        'CCR': acc,
        '1-off': off1,
        'Confusion matrix': conf_mat
    }

    return metrics


def _compute_sensitivities(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

	sum = np.sum(conf_mat, axis=1)
	mask = np.eye(conf_mat.shape[0], conf_mat.shape[1])
	correct = np.sum(conf_mat * mask, axis=1)
	sensitivities = correct / sum

	sensitivities = sensitivities[~np.isnan(sensitivities)]

	return sensitivities


def minimum_sensitivity(y_true, y_pred, labels=None):
	return np.min(_compute_sensitivities(y_true, y_pred, labels=labels))


def accuracy_off1(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
	n = conf_mat.shape[0]
	mask = np.eye(n, n) + np.eye(n, n, k=1), + np.eye(n, n, k=-1)
	correct = mask * conf_mat

	return 1.0 * np.sum(correct) / np.sum(conf_mat)


def print_metrics(metrics):
    print("")
    print('Confusion matrix :\n{}'.format(metrics['Confusion matrix']))
    print("")
    print('MS: {:.4f}'.format(metrics['MS']))
    print("")
    print('QWK: {:.4f}'.format(metrics['QWK']))
    print("")
    print('MAE: {:.4f}'.format(metrics['MAE']))
    print("")
    print('CCR: {:.4f}'.format(metrics['CCR']))
    print("")
    print('1-off: {:.4f}'.format(metrics['1-off']))

## Training process

In [7]:
def train(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    H: dict,
    num_classes: int,
):  # H: dict
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Train progress")
    model.train()
    mean_loss, accuracy = 0, 0
    y_pred, y_true = None, None

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Inputs and labels to device

        # Compute prediction error and accuracy of the training process
        pred = model(X)
        print(pred)
        print(y)
        loss = loss_fn(pred, y)

        mean_loss += loss
        accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Stack predictions and true labels to determine the confusion matrix
        pred_np = pred.argmax(1).cpu().detach().numpy()
        true_np = y.cpu().detach().numpy()
        if y_pred is None:
            y_pred = pred_np
        else:
            y_pred = np.concatenate((y_pred, pred_np))

        if y_true is None:
            y_true = true_np
        else:
            y_true = np.concatenate((y_true, true_np))

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy)
        progress_bar.update(1)

    accuracy /= size
    mean_loss /= num_batches

    H["train_loss"].append(loss.cpu().detach().numpy())
    H["train_acc"].append(accuracy)

    # Confusion matrix for training
    labels = range(0, num_classes)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    print("")
    print("Train Confusion matrix :\n{}".format(conf_mat))
    print("")

    return accuracy, mean_loss

## Testing process

In [8]:
def test(
    test_dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: torch.device,
    num_classes: int,
):
    num_batches = len(test_dataloader)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Test progress")
    model.eval()
    test_loss = 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)  # inputs and labels to device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # Stack predictions and true labels
            pred_np = pred.argmax(1).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

            # Update progress bar
            progress_bar.set_postfix(loss=test_loss / (batch + 1))
            progress_bar.update(1)

    test_loss /= num_batches
    metrics = compute_metrics(y_true, y_pred, num_classes)
    print_metrics(metrics)

    return metrics, test_loss

## Validation process

In [10]:
def validate(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: torch.device,
    H: dict,
    num_classes: int,
):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    loss, accuracy = 0, 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss += loss_fn(pred, y)
            accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

            pred_np = pred.argmax(1).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

    accuracy /= size
    loss /= num_batches

    H["val_loss"].append(loss.cpu().detach().numpy())
    H["val_acc"].append(accuracy)

    metrics = compute_metrics(y_true, y_pred, num_classes)

    return metrics, accuracy, loss

## Results

In [11]:
H = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

# To store validation metrics
validation_metrics = {}

# Definition to store best model weights
best_model_weights = model.state_dict()
best_qwk = 0.0

# Start time
start_time = time.time()

for e in range(optimiser_params["epochs"]):
    train_acc, train_loss = train(
        train_dataloader, model, loss_fn, optimizer, device, H, num_classes=num_classes
    )
    validation_metrics, val_acc, val_loss = validate(
        val_dataloader, model, loss_fn, device, H, num_classes=num_classes
    )

    if validation_metrics["QWK"] >= best_qwk:
        best_qwk = validation_metrics["QWK"]
        best_model_weights = deepcopy(model.state_dict())

    print("[INFO] EPOCH: {}/{}".format(e + 1, optimiser_params["epochs"]))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(train_loss, train_acc))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(val_loss, val_acc))

# Store last train error
train_error = H["train_loss"][-1]

# Restore best weights
model.load_state_dict(best_model_weights)

# Start evaluation
print("[INFO] Network evaluation ...")

test_metrics, test_loss = test(
    test_dataloader, model, loss_fn, device, num_classes=num_classes
)

# End time
end_time = time.time()
print("\n[INFO] Total training time: {:.2f}s".format(end_time - start_time))

Train progress:  25%|██████▌                   | 1/4 [00:01<00:03,  1.30s/it, accuracy=40, loss=257]

tensor([[0.4492, 0.5579, 0.4470, 0.4841, 0.3665],
        [0.4532, 0.5256, 0.4102, 0.5699, 0.3539],
        [0.5053, 0.4867, 0.3301, 0.5264, 0.3328],
        [0.4698, 0.4144, 0.3409, 0.5315, 0.3169],
        [0.4352, 0.5462, 0.3624, 0.5480, 0.4264],
        [0.4942, 0.4555, 0.4144, 0.5377, 0.4174],
        [0.4670, 0.5303, 0.3339, 0.5341, 0.4295],
        [0.4240, 0.4850, 0.4000, 0.5697, 0.4508],
        [0.4856, 0.4541, 0.3190, 0.5549, 0.4277],
        [0.4892, 0.4759, 0.4013, 0.5511, 0.4069],
        [0.4301, 0.5113, 0.2979, 0.5705, 0.4599],
        [0.4663, 0.4816, 0.3242, 0.5726, 0.3543],
        [0.4330, 0.4827, 0.4093, 0.5249, 0.3795],
        [0.4500, 0.5189, 0.3577, 0.5378, 0.3315],
        [0.4694, 0.4955, 0.4477, 0.6006, 0.4469],
        [0.4919, 0.5195, 0.3839, 0.5123, 0.4330],
        [0.4735, 0.5546, 0.3361, 0.5539, 0.4283],
        [0.4160, 0.5800, 0.3415, 0.5834, 0.3739],
        [0.4517, 0.5404, 0.3815, 0.5270, 0.4242],
        [0.4637, 0.4951, 0.3088, 0.5321, 0.4279],


Train progress:  75%|███████████████████▌      | 3/4 [00:01<00:00,  2.63it/s, accuracy=88, loss=190]

tensor([[0.8670, 0.8377, 0.8199, 0.7870, 0.8756],
        [0.5322, 0.4781, 0.3185, 0.2297, 0.0717],
        [0.7961, 0.7365, 0.6125, 0.5568, 0.5607],
        [0.6071, 0.5901, 0.2925, 0.2243, 0.0806],
        [0.6930, 0.6910, 0.4381, 0.3954, 0.2443],
        [0.5751, 0.5518, 0.3515, 0.2160, 0.0839],
        [0.5225, 0.5231, 0.3085, 0.1822, 0.0406],
        [0.6819, 0.6709, 0.4801, 0.4058, 0.2140],
        [0.5956, 0.4954, 0.2812, 0.2468, 0.1328],
        [0.8609, 0.7968, 0.6201, 0.6801, 0.7087],
        [0.9327, 0.9191, 0.8970, 0.8866, 0.9702],
        [0.5906, 0.5694, 0.3025, 0.2218, 0.0674],
        [0.6689, 0.5966, 0.4917, 0.4296, 0.1598],
        [0.6784, 0.6160, 0.4300, 0.4005, 0.1774],
        [0.5653, 0.5591, 0.2848, 0.1883, 0.0314],
        [0.4993, 0.4327, 0.1901, 0.1223, 0.0543],
        [0.5330, 0.5213, 0.2139, 0.1304, 0.0261],
        [0.6350, 0.6538, 0.4170, 0.3384, 0.1851],
        [0.6001, 0.5095, 0.2835, 0.2366, 0.1063],
        [0.8027, 0.7906, 0.7561, 0.6186, 0.5467],


Train progress: 100%|█████████████████████████| 4/4 [00:01<00:00,  2.37it/s, accuracy=95, loss=69.8]

...

Train progress: 100%|█████████████████████████| 4/4 [00:00<00:00,  4.21it/s, accuracy=71, loss=23.7]


Train Confusion matrix :
[[ 71   0   0   0   0   0]
 [205   0   0   0   0   0]
 [110   0   0   0   0   0]
 [145   0   0   0   0   0]
 [101   0   0   0   0   0]
 [ 48   0   0   0   0   0]]



[INFO] EPOCH: 5/5
Train loss: 64.181602, Train accuracy: 0.1044
Val loss: 100.911156, Val accuracy: 0.0992

[INFO] Network evaluation ...


Test progress: 100%|████████████████████████████████████████| 2/2 [00:00<00:00,  3.50it/s, loss=127]


Confusion matrix :
[[ 2 24  0  0  0  0]
 [ 3 58  0  0  0  0]
 [ 0 34  0  0  0  0]
 [ 0 39  0  0  0  0]
 [ 0 29  0  0  0  0]
 [ 0 12  0  0  0  0]]

MS: 0.0000

QWK: 0.0219

MAE: 1.3632

CCR: 0.2985

1-off: 0.6020

[INFO] Total training time: 8.83s
